<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Explore-other-reward-functions" data-toc-modified-id="Explore-other-reward-functions-1">Explore other reward functions</a></span><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1.1">Goal</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2">Imports</a></span></li><li><span><a href="#Code" data-toc-modified-id="Code-1.3">Code</a></span><ul class="toc-item"><li><span><a href="#Data-generation" data-toc-modified-id="Data-generation-1.3.1">Data generation</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1.3.2">Evaluation</a></span></li></ul></li><li><span><a href="#risk_averse_goose" data-toc-modified-id="risk_averse_goose-1.4">risk_averse_goose</a></span><ul class="toc-item"><li><span><a href="#Generation" data-toc-modified-id="Generation-1.4.1">Generation</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.5">Summary</a></span></li></ul></li></ul></div>

# Explore other reward functions

## Goal

The goal of this notebooks is to continue learning about the Q value function, but trying other reward methods.

This is a continuation of notebook 004, which had become too big. In this notebook I will split the functionality
of playing games and generating data from training from those games.

## Imports

In [1]:
# Use this to reload changes in python scripts
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import kaggle_environments
from kaggle_environments import make
from kaggle_environments import evaluate
from kaggle_environments.envs.hungry_geese.hungry_geese import Action
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import random
import pandas as pd
from tqdm.notebook import tqdm
import time
from itertools import permutations
from functools import partial
import gc
from concurrent.futures import ProcessPoolExecutor
import json
import logging

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
import tensorflow.keras as keras

from hungry_geese import GameState
from hungry_geese.state import make_board_egocentric, get_head_position, combine_data, horizontal_simmetry, vertical_simmetry
from hungry_geese.utils import ACTIONS, opposite_action, get_timestamp, log_ram_usage, configure_logging
from hungry_geese.definitions import ACTION_TO_IDX
from hungry_geese.agents import EpsilonAgent
from hungry_geese.evaluation import play_matches_in_parallel, monitor_progress
from hungry_geese.elo import EloRanking
from hungry_geese.model import simple_model, create_model_for_training

%config Completer.use_jedi = False

Loading environment football failed: No module named 'gfootball'


In [3]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (30, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

configure_logging()
logger = logging.getLogger(__name__)

## Code

### Data generation

In [4]:
def play_matches_in_parallel_and_save_history(agents, max_workers=20, n_matches=1000,
                                              running_on_notebook=True):
    """
    Plays n_matches in parallel using ProcessPoolExecutor

    Parameters
    -----------
    agents : list
        List of the agents that we will use for playing
    """
    log_ram_usage()
    output_path = get_games_output_path(agents, n_matches)
    assert not os.path.exists(output_path), 'Output path already exists: %s' % output_path
    with ProcessPoolExecutor(max_workers=max_workers) as pool:
        matches_results = []
        submits = []
        for i in range(n_matches):
            submits.append(pool.submit(play_game, agents=agents))
        monitor_progress(submits, running_on_notebook)
        matches_results = [submit.result() for submit in submits]
    
    log_ram_usage()
    logger.info('saving matches to json: %s' % output_path)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(matches_results, f)
        
    del submits
    del matches_results
    gc.collect()
    log_ram_usage()

def play_game(agents):
    env = make("hungry_geese")
    return env.run(agents=agents)

def get_games_output_path(agents, n_matches):
    output_path = '/mnt/hdd0/Kaggle/hungry_geese/games/%s_vs_%s_matches_%i.json' % (
        os.path.splitext(os.path.basename(agents[0]))[0], 
        os.path.splitext(os.path.basename(agents[1]))[0], 
        n_matches)
    return output_path

In [5]:
def test_agent(agent):
    print('Testing agent: %s' % agent)
    env = make('hungry_geese', debug=True)
    env.run([agent]*4)

In [6]:
def create_train_data(agents, n_matches, reward_name, agent_idx_range=None):
    """
    Creates train data without any simmetry
    
    Parameters
    ----------
    agents : list of str
        Names or paths of the agents
    n_matches : int
        Number of matches to play
    reward_name : str
        Name of the reward function that we want to use
    max_workers : int
    agent_idx_range : list of int
        Idx of the agents we want to use for collecting data, if None all the agents
        will be used
    """
    output_path = get_games_output_path(agents, n_matches)
    if not os.path.exists(output_path):
        play_matches_in_parallel_and_save_history(agents, n_matches=n_matches)
    else:
        logger.info('Matches already saved in disk, proceed to loading')
    with open(output_path, 'r') as f:
        matches_results = json.load(f)
    
    log_ram_usage()
    env = make("hungry_geese")
    conf = env.configuration

    state = GameState(reward_name=reward_name)
    train_data = []
    agent_idx_range = agent_idx_range or list(range(4))
    for _ in tqdm(range(len(matches_results)), desc='Creating game data'):
        match = matches_results[0]
        for idx in agent_idx_range:
            state.reset()
            for step_idx, step in enumerate(match):
                observation = step[0]['observation'].copy()
                observation['index'] = idx
                state.update(observation, conf)
                if step_idx:
                    state.add_action(step[idx]['action'])
                if not observation['geese'][idx]:
                    break
            train_data.append(state.prepare_data_for_training())
        del matches_results[0]
        
    log_ram_usage()
    train_data = combine_data(train_data)
    logger.info('Size of the boards is %.1f GB' % (train_data[0].nbytes/1e9))
    output_file = get_data_output_path(agents, n_matches, reward_name)
    logger.info('Saving data on: %s' % output_file)
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    np.savez_compressed(output_file, boards=train_data[0], features=train_data[1], actions=train_data[2], rewards=train_data[3])
    del state
    del train_data
    gc.collect()
    log_ram_usage()
    
def get_data_output_path(agents, n_matches, reward_name):
    output_file = '/mnt/hdd0/Kaggle/hungry_geese/data/%s/%s_vs_%s_matches_%i.npz' % (
        reward_name,
        os.path.splitext(os.path.basename(agents[0]))[0], 
        os.path.splitext(os.path.basename(agents[1]))[0], 
        n_matches)
    return output_file

### Evaluation

In [7]:
initial_elo_ranking = pd.read_csv('../data/elo_ranking.csv', index_col='model')
initial_elo_ranking

,ranking
model,
boilergoose,1269
besthoarder,1227
crazy_goose,1186
risk_averse_goose,1176
goose_on_a_healthy_diet,1157
straightforward_bfs,1004
greedy,981
greedyhoarder,766
random_plus,706


In [8]:
agents = {
    'random': 'random',
    'random_plus': '../forum/agents/random_plus.py',
    'greedy': 'greedy',
    'risk_averse_goose': '../forum/agents/risk_averse_goose.py',
    'goose_on_a_healthy_diet': '../forum/agents/goose_on_a_healthy_diet.py',
    'straightforward_bfs': '../forum/agents/straightforward_bfs.py',
    'boilergoose': '../forum/agents/boilergoose.py',
    'crazy_goose': '../forum/agents/crazy_goose.py',
    'besthoarder': '../forum/agents/besthoarder.py',
    'greedyhoarder': '../forum/agents/greedyhoarder.py',
    'q_value': '/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/forum/agents/random_plus.py',
}

In [9]:
def evaluate_agent(new_agent, adversary_agents, n_matches, single_agent=True, max_workers=20, run_agent_test=False):
    agent_name = list(new_agent.keys())[0]
    # I have found that for certain agents if the test is run then evaluation is frozen
    if run_agent_test: test_agent(new_agent[agent_name])
    reduced_agents_set = new_agent.copy()
    for adversary_agent in adversary_agents:
        reduced_agents_set[adversary_agent] = agents[adversary_agent]
    
    if single_agent:
        sample_agents_func = lambda: [agent_name] + np.random.choice(adversary_agents, 3).tolist()
    else:
        def sample_agents_func():
            while 1:
                sampled_agents = [agent_name] + np.random.choice(adversary_agents + [agent_name], 3).tolist()
                if len(np.unique(sampled_agents)) >=2:
                    break
            return sampled_agents

    matches_agents, matches_results = play_matches_in_parallel(reduced_agents_set, sample_agents_func, n_matches=n_matches, max_workers=max_workers)
    return matches_agents, matches_results, reduced_agents_set
    
def compute_elo_ranking(matches_agents, matches_results, reduced_agents_set):
    initial_agent_elo = 1000
    for k in [32, 16, 8, 4, 2, 1]:
        agent_name = [name for name in reduced_agents_set if name not in initial_elo_ranking][0]
        initial_ranking = initial_elo_ranking.to_dict()['ranking']
        initial_ranking[agent_name] = initial_agent_elo
        initial_ranking = {key: initial_ranking[key] for key in reduced_agents_set}
        elo_ranking = EloRanking(initial_ranking, {agent_name}, k=k)
        for match_agents, match_results in zip(matches_agents, matches_results):
            elo_ranking.add_match(match_agents, match_results)
        initial_agent_elo = elo_ranking.summary().loc[agent_name, 'ranking']
    elo_ranking.plot()
    return elo_ranking.summary()

## risk_averse_goose

### Generation

In [19]:
n_matches = 100
agents = ['/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/forum/agents/epsilon_risk_averse_goose_002.py'] + \
    ['/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/forum/agents/risk_averse_goose.py']*3
reward_name = 'ranking_reward_-4_4'
create_train_data(agents=agents, n_matches=n_matches, reward_name=reward_name, agent_idx_range=[0])

2021-03-20 10:50:40,564 - hungry_geese.utils - INFO - ram_memory used: 0.41 GB	 available: 54.63 GB


  0%|          | 0/100 [00:00<?, ?it/s]

2021-03-20 10:50:47,992 - hungry_geese.utils - INFO - ram_memory used: 0.51 GB	 available: 54.62 GB
2021-03-20 10:50:47,993 - __main__ - INFO - saving matches to json
2021-03-20 10:50:49,121 - hungry_geese.utils - INFO - ram_memory used: 0.42 GB	 available: 54.66 GB
2021-03-20 10:50:49,445 - hungry_geese.utils - INFO - ram_memory used: 0.48 GB	 available: 54.60 GB


Creating game data:   0%|          | 0/100 [00:00<?, ?it/s]

2021-03-20 10:50:50,266 - hungry_geese.utils - INFO - ram_memory used: 0.44 GB	 available: 54.67 GB
2021-03-20 10:50:50,287 - __main__ - INFO - Size of the boards is 0.1 GB
2021-03-20 10:50:50,288 - __main__ - INFO - Saving data on: /mnt/hdd0/Kaggle/hungry_geese/data/ranking_reward_-4_4/epsilon_risk_averse_goose_002_vs_risk_averse_goose_matches_100.npz
2021-03-20 10:50:50,687 - hungry_geese.utils - INFO - ram_memory used: 0.42 GB	 available: 54.67 GB


In [29]:
n_matches = 8000
agents = ['/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/forum/agents/epsilon_risk_averse_goose_002.py'] + \
    ['/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/forum/agents/risk_averse_goose.py']*3
reward_name = 'ranking_reward_-4_4'
create_train_data(agents=agents, n_matches=n_matches, reward_name=reward_name, agent_idx_range=[0])

2021-03-20 10:52:12,262 - hungry_geese.utils - INFO - ram_memory used: 0.42 GB	 available: 54.66 GB


  0%|          | 0/8000 [00:00<?, ?it/s]

2021-03-20 11:00:17,852 - hungry_geese.utils - INFO - ram_memory used: 8.34 GB	 available: 46.54 GB
2021-03-20 11:00:17,853 - __main__ - INFO - saving matches to json: /mnt/hdd0/Kaggle/hungry_geese/games/epsilon_risk_averse_goose_002_vs_risk_averse_goose_matches_8000.json
2021-03-20 11:01:43,184 - hungry_geese.utils - INFO - ram_memory used: 0.43 GB	 available: 54.45 GB
2021-03-20 11:02:07,252 - hungry_geese.utils - INFO - ram_memory used: 5.95 GB	 available: 48.92 GB


Creating game data:   0%|          | 0/8000 [00:00<?, ?it/s]

2021-03-20 11:03:07,641 - hungry_geese.utils - INFO - ram_memory used: 6.20 GB	 available: 48.66 GB
2021-03-20 11:03:08,638 - __main__ - INFO - Size of the boards is 4.3 GB
2021-03-20 11:03:08,639 - __main__ - INFO - Saving data on: /mnt/hdd0/Kaggle/hungry_geese/data/ranking_reward_-4_4/epsilon_risk_averse_goose_002_vs_risk_averse_goose_matches_8000.npz
2021-03-20 11:03:33,164 - hungry_geese.utils - INFO - ram_memory used: 4.68 GB	 available: 50.20 GB


In [ ]:
n_matches = 40000
agents = ['/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/forum/agents/epsilon_risk_averse_goose_002.py'] + \
    ['/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/forum/agents/risk_averse_goose.py']*3
reward_name = 'ranking_reward_-4_4'
create_train_data(agents=agents, n_matches=n_matches, reward_name=reward_name, agent_idx_range=[0])

2021-03-20 11:04:07,736 - hungry_geese.utils - INFO - ram_memory used: 0.34 GB	 available: 55.21 GB


  0%|          | 0/40000 [00:00<?, ?it/s]

## Summary